In [20]:
import pandas as pd
import requests
import edgar_functions as edgar
headers = {"User-Agent": "victorbenitogr@gmail.com"}  # Need to add your email address here
import os

In [21]:
def get_facts(ticker, headers=headers):
    cik = edgar.cik_matching_ticker(ticker)
    url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    company_facts = requests.get(url, headers=headers).json()
    return company_facts

In [22]:
facts = get_facts("WSM")
facts["facts"]["us-gaap"]["AccountsPayableCurrent"]['units']['USD']

[{'end': '2009-08-02',
  'val': 129798000,
  'accn': '0001193125-10-208171',
  'fy': 2010,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2010-09-10',
  'frame': 'CY2009Q2I'},
 {'end': '2009-11-01',
  'val': 171904000,
  'accn': '0001193125-10-278135',
  'fy': 2010,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2010-12-10',
  'frame': 'CY2009Q3I'},
 {'end': '2010-01-31',
  'val': 188241000,
  'accn': '0001193125-10-208171',
  'fy': 2010,
  'fp': 'Q2',
  'form': '10-Q',
  'filed': '2010-09-10'},
 {'end': '2010-01-31',
  'val': 188241000,
  'accn': '0001193125-10-278135',
  'fy': 2010,
  'fp': 'Q3',
  'form': '10-Q',
  'filed': '2010-12-10'},
 {'end': '2010-01-31',
  'val': 188241000,
  'accn': '0001193125-11-085046',
  'fy': 2010,
  'fp': 'FY',
  'form': '10-K',
  'filed': '2011-03-31',
  'frame': 'CY2009Q4I'},
 {'end': '2010-05-02',
  'val': 177938000,
  'accn': '0001193125-11-162733',
  'fy': 2011,
  'fp': 'Q1',
  'form': '10-Q',
  'filed': '2011-06-10',
  'frame': 'CY2010Q1I'},
 {'end': '

In [23]:
def facts_DF(ticker, headers=headers):
    facts = get_facts(ticker, headers)
    us_gaap_data = facts["facts"]["us-gaap"]
    df_data = []
    for fact, details in us_gaap_data.items():
        for unit in details["units"]:
            for item in details["units"][unit]:
                row = item.copy()
                row["fact"] = fact
                df_data.append(row)

    df = pd.DataFrame(df_data)
    df["end"] = pd.to_datetime(df["end"])
    df["start"] = pd.to_datetime(df["start"])
    df = df.drop_duplicates(subset=["fact", "end", "val"])
    df.set_index("end", inplace=True)
    labels_dict = {fact: details["label"] for fact, details in us_gaap_data.items()}
    return df, labels_dict

In [24]:
facts, label_dict = facts_DF("WSM")

In [25]:
def annual_facts(ticker, headers=headers):
    accession_nums = edgar.get_filtered_filings(
        ticker, ten_k=True, just_accession_numbers=True
    )
    df, label_dict = facts_DF(ticker, headers)
    ten_k = df[df["accn"].isin(accession_nums)]
    ten_k = ten_k[ten_k.index.isin(accession_nums.index)]
    pivot = ten_k.pivot_table(values="val", columns="fact", index="end")
    pivot.rename(columns=label_dict, inplace=True)
    return pivot.T

In [26]:
def quarterly_facts(ticker, headers=headers):
    accession_nums = get_filtered_filings(
        ticker, ten_k=False, just_accession_numbers=True
    )
    df, label_dict = facts_DF(ticker, headers)
    ten_q = df[df["accn"].isin(accession_nums)]
    ten_q = ten_q[ten_q.index.isin(accession_nums.index)].reset_index(drop=False)
    ten_q = ten_q.drop_duplicates(subset=["fact", "end"], keep="last")
    pivot = ten_q.pivot_table(values="val", columns="fact", index="end")
    pivot.rename(columns=label_dict, inplace=True)
    return pivot.T

In [27]:
quarterly = quarterly_facts('WSM')

NameError: name 'get_filtered_filings' is not defined

In [ ]:
def save_dataframe_to_csv(dataframe, folder_name, ticker, statement_name, frequency):
    directory_path = os.path.join(folder_name, ticker)
    os.makedirs(directory_path, exist_ok=True)
    file_path = os.path.join(directory_path, f"{statement_name}_{frequency}.csv")
    dataframe.to_csv(file_path)
    return None